<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-11 10:23:10
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: -8.55 K (-0.06%)
Current PnL: -29.39 L (-18.99%)
CY Booked + Current PnL: -15.10 L (-9.76%)
-------------------
Total profit:  1.20 L
Total loss:  -30.59 L
-------------------
Total Booked + Current PnL: 11.88 L (9.29%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.29%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 94.02 L (67.7%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.65%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.61,-17.90,21.91,0.09,18128.0,-18037.0,82740.0,92.10,35.0,M-SC,2.34,253.0,-0.99,0.62,8.70,OX40N,NTT,DURABLES
50,MASFIN,398.61,-0.56,-2.77,25.53,22.05,24321.0,-2715.0,95265.0,-15.99,54.0,H-SC,7.06,164.0,-0.11,0.71,38.47,XR,ATH,FINANCE
43,ITC,452.00,-0.33,-1.83,12.49,10.44,24540.0,-3658.0,196480.0,-41.15,45.0,X-LC,1.76,5.0,-0.15,1.47,3.63,X40,NTT,FMCG
39,INDIGOPNTS,1408.00,-1.98,-14.59,17.11,0.02,25508.0,-25474.0,149085.0,127.12,47.0,M-SC,16.10,234.0,-1.00,1.11,29.68,OX40N,NTT,PAINTS
82,VOLTAS,1530.00,0.33,5.24,13.73,19.69,27706.0,10053.0,201795.0,-5.08,46.0,X-MC,2.55,78.0,0.36,1.51,12.40,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,3.66,-17.78,47.87,21.58,55159.0,-24912.0,115227.0,-58.54,26.0,X-MC,7.13,56.0,-0.45,0.86,3.66,X40N,ATH,IT
67,SONACOMS,806.63,2.66,-16.25,66.62,39.54,56439.0,-16441.0,84718.0,-34.50,47.0,M-SC,4.57,220.0,-0.29,0.63,19.46,AR,ATH,AUTO
1,ABB,7934.00,1.38,-4.17,51.91,45.57,130140.0,-10915.0,250704.0,-38.46,56.0,H-MC,5.10,120.0,-0.08,1.87,6.68,AR,NTT,ELECTRICAL
62,SATIN,274.00,1.38,-20.49,90.97,51.83,199832.0,-56616.0,219668.0,-29.04,42.0,H-SC,2.18,146.0,-0.28,1.64,7.01,XY24,NTT,FINANCE
69,SURYODAY,216.00,1.35,-22.82,59.41,23.03,82111.0,-40861.0,138210.0,51.32,42.0,H-SC,10.08,167.0,-0.50,1.03,37.13,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,-2.37,-4.49,26.84,21.14,58275.0,-10219.0,217120.0,-11.18,38.0,X-MC,4.15,74.0,-0.18,1.62,22.38,XY24,NTT,PAINTS
18,COFFEEDAY,80.0,-2.27,-46.23,151.57,35.27,92543.0,-52493.0,61056.0,-57.75,25.0,L-SC,10.89,270.0,-0.57,0.46,49.44,XR,NTT,HOTELS
39,INDIGOPNTS,1408.0,-1.98,-14.59,17.11,0.02,25508.0,-25474.0,149085.0,127.12,47.0,M-SC,16.10,234.0,-1.00,1.11,29.68,OX40N,NTT,PAINTS
44,JCHAC,2282.0,-1.62,-36.79,58.20,-0.01,47014.0,-47025.0,80780.0,16421.74,23.0,M-SC,4.99,233.0,-1.00,0.60,3.39,OX40N,NTT,AC
16,CERA,9475.0,-1.33,-30.90,79.30,23.89,96384.0,-54359.0,121544.0,-33.41,31.0,H-SC,9.73,157.0,-0.56,0.91,4.97,OX40N,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.99,-7.12,123.86,107.91,169846.0,-10517.0,137127.0,-24.60,38.0,M-SC,12.37,216.0,-0.06,1.02,2.24,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.98,-14.59,17.11,0.02,25508.0,-25474.0,149085.0,127.12,47.0,M-SC,16.10,234.0,-1.00,1.11,29.68,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.61,-17.90,21.91,0.09,18128.0,-18037.0,82740.0,92.10,35.0,M-SC,2.34,253.0,-0.99,0.62,8.70,OX40N,NTT,DURABLES
66,SIS,528.00,-0.72,-25.42,62.59,21.26,51831.0,-28221.0,82811.0,1951.81,43.0,H-SC,4.61,166.0,-0.54,0.62,11.89,OX40N,NTT,MISC
47,KANSAINER,340.00,-1.20,-25.93,53.20,13.47,106260.0,-69930.0,199737.0,-69.48,29.0,H-SC,4.21,158.0,-0.66,1.49,1.56,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.48,6.32,62.02,72.27,111738.0,10717.0,180165.0,-8.8,66.0,M-LC,1.93,99.0,0.1,1.35,12.89,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.48,6.32,62.02,72.27,111738.0,10717.0,180165.0,-8.80,66.0,M-LC,1.93,99.0,0.10,1.35,12.89,XR,NTT,IT
38,INDIAMART,4810.62,0.10,-3.88,115.06,106.72,136411.0,-4780.0,118556.0,-52.34,31.0,H-SC,1.99,139.0,-0.04,0.89,18.60,AR,ATH,MISC
25,FINCABLES,1641.55,-0.99,-7.12,123.86,107.91,169846.0,-10517.0,137127.0,-24.60,38.0,M-SC,12.37,216.0,-0.06,1.02,2.24,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,0.11,-2.50,39.11,35.63,79916.0,-5237.0,204337.0,-16.30,39.0,H-MC,3.15,119.0,-0.07,1.53,13.76,AR,ATH,PHARMA
1,ABB,7934.00,1.38,-4.17,51.91,45.57,130140.0,-10915.0,250704.0,-38.46,56.0,H-MC,5.10,120.0,-0.08,1.87,6.68,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.18,-21.43,88.94,48.45,140114.0,-42968.0,157538.0,-27.18,25.0,X-MC,12.18,64.0,-0.31,1.18,2.57,X40N,ATH,FINANCE
22,DIXON,18931.72,3.66,-17.78,47.87,21.58,55159.0,-24912.0,115227.0,-58.54,26.0,X-MC,7.13,56.0,-0.45,0.86,3.66,X40N,ATH,IT
75,TMPV,600.00,0.44,-18.27,73.96,42.18,167082.0,-50501.0,225909.0,-26.74,27.0,X-LC,6.07,3.0,-0.30,1.69,0.44,XY24,NTT,AUTO
33,HINDUNILVR,2922.00,-0.05,-10.51,27.02,13.67,62160.0,-27013.0,230050.0,-17.28,28.0,X-LC,2.63,14.0,-0.43,1.72,8.60,XY25,NTT,FMCG
53,PAGEIND,51605.07,0.45,-9.95,40.00,26.07,58976.0,-16300.0,147440.0,-33.64,30.0,X-MC,12.16,55.0,-0.28,1.10,0.45,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,1.31,-80.43,537.57,24.79,264995.0,-202561.0,49295.0,-81.32,41.0,X-SC,1.21,86.0,-0.76,0.37,20.51,X40N,NTT,MISC
8,AWL,485.0,-0.34,-22.05,96.91,53.50,227948.0,-66535.0,235216.0,-62.23,32.0,X-MC,1.72,58.0,-0.29,1.76,4.48,XY24,NTT,FMCG
43,ITC,452.0,-0.33,-1.83,12.49,10.44,24540.0,-3658.0,196480.0,-41.15,45.0,X-LC,1.76,5.0,-0.15,1.47,3.63,X40,NTT,FMCG
36,ICICIPRULI,790.0,-0.56,5.98,23.58,30.97,44165.0,10562.0,187300.0,-18.97,62.0,X-MC,2.05,75.0,0.24,1.40,19.29,X40,ATH,INSURANCE
3,ACC,3906.0,-0.29,-25.32,119.80,64.15,212897.0,-60241.0,177710.0,-56.31,34.0,X-SC,2.11,82.0,-0.28,1.33,0.00,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.29,-25.32,119.80,64.15,212897.0,-60241.0,177710.0,-56.31,34.0,X-SC,2.11,82.0,-0.28,1.33,0.00,XY24,BTT,CEMENT
54,PGHH,17907.65,-0.63,-6.48,43.03,33.76,80810.0,-13020.0,187800.0,-33.99,31.0,X-MC,5.25,57.0,-0.16,1.40,0.00,X40,ATH,FMCG
31,HAVELLS,2069.16,0.07,-10.95,48.76,32.48,142423.0,-35902.0,292089.0,-15.47,35.0,X-MC,5.52,67.0,-0.25,2.18,0.07,X40,ATH,ELECTRICAL
75,TMPV,600.00,0.44,-18.27,73.96,42.18,167082.0,-50501.0,225909.0,-26.74,27.0,X-LC,6.07,3.0,-0.30,1.69,0.44,XY24,NTT,AUTO
53,PAGEIND,51605.07,0.45,-9.95,40.00,26.07,58976.0,-16300.0,147440.0,-33.64,30.0,X-MC,12.16,55.0,-0.28,1.10,0.45,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.12,-30.60,105.82,42.84,47742.0,-19890.0,45116.0,-54.82,39.0,X-SC,36.56,83.0,-0.42,0.34,2.07,XY24,NTT,MISC
15,CAMS,4762.00,1.31,-80.43,537.57,24.79,264995.0,-202561.0,49295.0,-81.32,41.0,X-SC,1.21,86.0,-0.76,0.37,20.51,X40N,NTT,MISC
59,RELAXO,1176.00,-0.72,-47.06,187.71,52.31,144246.0,-68315.0,76845.0,-43.66,45.0,X-SC,6.40,91.0,-0.47,0.57,2.86,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.08,-37.95,117.88,35.20,94122.0,-48824.0,79846.0,4.69,34.0,X-SC,15.67,92.0,-0.52,0.60,1.42,X40,NTT,FOOTWEAR
22,DIXON,18931.72,3.66,-17.78,47.87,21.58,55159.0,-24912.0,115227.0,-58.54,26.0,X-MC,7.13,56.0,-0.45,0.86,3.66,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.31,-13.61,38.08,19.28,113807.0,-47092.0,298864.0,-24.60,57.0,X-LC,5.22,1.0,-0.41,2.23,10.49,X40,ATH,IT
41,INFY,2275.00,0.40,8.32,42.97,54.87,145637.0,26038.0,338926.0,-15.65,61.0,X-LC,3.22,2.0,0.18,2.53,17.29,X40,BTT,IT
75,TMPV,600.00,0.44,-18.27,73.96,42.18,167082.0,-50501.0,225909.0,-26.74,27.0,X-LC,6.07,3.0,-0.30,1.69,0.44,XY24,NTT,AUTO
81,VBL,671.64,-0.88,-5.30,43.27,35.67,129418.0,-16748.0,299094.0,-16.55,50.0,X-LC,4.62,4.0,-0.13,2.23,7.82,X40N,ATH,FMCG
43,ITC,452.00,-0.33,-1.83,12.49,10.44,24540.0,-3658.0,196480.0,-41.15,45.0,X-LC,1.76,5.0,-0.15,1.47,3.63,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.16,-17.91,121.32,81.67,93421.0,-16806.0,77004.0,6916.67,39.0,L-SC,13.84,271.0,-0.18,0.57,49.12,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.56,-2.77,25.53,22.05,24321.0,-2715.0,95265.0,-15.99,54.0,H-SC,7.06,164.0,-0.11,0.71,38.47,XR,ATH,FINANCE
13,BSOFT,831.70,1.21,-22.37,94.23,50.79,102891.0,-31462.0,109191.0,-0.27,67.0,H-SC,5.86,151.0,-0.31,0.82,27.13,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.98,-14.59,17.11,0.02,25508.0,-25474.0,149085.0,127.12,47.0,M-SC,16.10,234.0,-1.00,1.11,29.68,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.26,-1.12,-3.08,24.78,20.95,60469.0,-7744.0,244024.0,-4.01,46.0,X-LC,18.64,31.0,-0.13,1.82,31.46,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.21,-22.37,94.23,50.79,102891.0,-31462.0,109191.0,-0.27,67.0,H-SC,5.86,151.0,-0.31,0.82,27.13,XR,ATH,IT
84,WIPRO,420.00,0.48,6.32,62.02,72.27,111738.0,10717.0,180165.0,-8.80,66.0,M-LC,1.93,99.0,0.10,1.35,12.89,XR,NTT,IT
56,RAJESHEXPO,518.00,0.68,-63.00,170.45,0.07,86512.0,-86422.0,50755.0,1759.85,57.0,L-SC,2.37,268.0,-1.00,0.38,26.23,OX40N,NTT,JEWELLERY
32,HCLTECH,1922.01,-0.38,7.70,15.81,24.73,41194.0,18633.0,260557.0,12.34,61.0,X-LC,6.17,7.0,0.45,1.95,24.11,X40,ATH,IT
65,SIEMENS,4671.50,0.14,-14.44,46.70,25.51,74356.0,-26875.0,159220.0,-2.24,47.0,H-LC,4.15,49.0,-0.36,1.19,17.33,AR,ATH,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.74
1,25,45.78
2,50,76.74


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.04
MC    30.33
LC    24.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.30
X40      22.46
X40N     10.83
XR       10.12
XY25      9.42
AR        9.41
OX40N     7.75
SR        1.02
MH        0.74
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.56
X-MC    23.54
X-LC    21.08
M-SC    12.10
X-SC     6.97
H-MC     5.02
M-MC     1.41
L-SC     1.41
M-LC     1.35
H-LC     1.19
L-LC     1.06
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.38,-8.17,43.16
IT,12.79,-17.77,81.44
FINANCE,9.60,-20.62,71.58
ELECTRICAL,6.25,-12.07,53.12
MISC,6.10,-59.33,124.61
PAINTS,6.04,-14.00,30.93
INSURANCE,4.64,-1.90,37.81
PHARMA,4.13,-1.10,33.46
AUTO,3.42,-25.38,79.13


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3248055.0,21
AR,1313637.0,10
XR,1308004.0,13
X40,1032045.0,14
X40N,998328.0,9
OX40N,753434.0,10
XY25,380194.0,6
SR,283887.0,2
MH,84897.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3692664.0,25
M-SC,1436291.0,15
X-MC,1331970.0,16
X-SC,987687.0,8
X-LC,940716.0,11
H-MC,406175.0,3
L-SC,272476.0,3
M-LC,111738.0,1
H-LC,74356.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1290880.0      6
           AR         902064.0      5
M-SC       XY24       817700.0      6
H-SC       XR         781677.0      7
X-SC       X40N       507193.0      3
X-MC       X40        462068.0      7
           XY24       400248.0      3
X-LC       X40        385647.0      5
H-SC       OX40N      349259.0      4
M-SC       OX40N      317663.0      5
X-SC       XY24       296164.0      3
H-SC       SR         283887.0      2
X-MC       X40N       283545.0      4
X-LC       XY24       246944.0      2
H-MC       AR         210056.0      2
X-LC       X40N       207590.0      2
H-MC       XY24       196119.0      1
X-MC       XY25       186109.0      2
L-SC       XR         185964.0      2
X-SC       X40        184330.0      2
M-SC       XR         173767.0      2
           AR         127161.0      2
M-LC       XR         111738.0      1
X-LC       XY25       100535.0      2
L-SC       OX40N       86512.0      1
H-SC       MH          84897.0      1
H-LC       AR          74356.0      1
M-MC       XY25        56287.0      1
L-MC       XR          54858.0      1
L-LC       XY25        37263.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
